In [6]:
import torch
import numpy as np
import pandas as pd

import util
import datasets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset_names = ["cifar10", "cifar100"]
models_folder = "../trained_models"

def run_epoch(model, train, test, device, in_features, flatten = False):
    criterion = torch.nn.CrossEntropyLoss()

    agg_acc_top1 = []
    agg_loss = []
    model.train()
    with torch.no_grad():
        for x, y in train:
            if flatten:
                x = x.view(-1, in_features)
            x, y = x.to(device).float(), y.to(device)
            pred = model(x)
            loss_value = criterion(pred, y)
            agg_acc_top1.append((pred.cpu().argmax(1) == y.cpu()).float().mean())
            agg_loss.append(loss_value.item())
        top1_train = round(np.mean(agg_acc_top1) * 100, 1)
        loss = round(np.mean(agg_loss), 4)

    agg_acc_top1 = []
    model.eval()
    with torch.no_grad():
        for x, y in test:
            if flatten:
                x = x.view(-1, in_features)
            x, y = x.to(device).float(), y.to(device)
            pred = model(x)
            agg_acc_top1.append((pred.cpu().argmax(1) == y.cpu()).float().mean())
        top1_test = round(np.mean(agg_acc_top1) * 100, 1)

    return {
        'Loss': round(loss, 3),
        'Top-1 Accuracy (train)': top1_train,
        'Top-1 Accuracy (test)': top1_test,
    }

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class ConvNet(torch.nn.Module):
    def __init__(self, conv_layers: list[int], linear_layers: list[int], linear_type: str, pcn_dims: int = None):
        super().__init__()

        assert len(conv_layers) >= 2, "Must be at least 2 convolutional layers"
        assert len(linear_layers) >= 1, "Must be at least 1 linear layer"
        assert linear_type in ["mlp", "pcn"], "linear_type must be either 'mlp' or 'pcn'"
        if linear_type == "pcn":
            assert pcn_dims is not None, "pcn_dims must be specified if linear_type is 'pcn'"
            
        L = [
            torch.nn.Conv2d(conv_layers[0], conv_layers[1], 7, padding=3, stride=2),
            torch.nn.ReLU(),
        ]
        c = conv_layers[1]
        for l in conv_layers[2:]:
            L.append(torch.nn.Conv2d(c, l, 3, padding=1, stride=2))
            L.append(torch.nn.ReLU())
            c = l
        self.net = torch.nn.Sequential(*L)
        self.classifier = util.MLP([c, *linear_layers]) if linear_type == "mlp" \
            else util.PCN([c, *linear_layers], pcn_dims)

    def forward(self, x: torch.Tensor):
        z = self.net(x)
        z = torch.amax(z, dim=[2, 3])
        return self.classifier(z)

In [7]:

# LINEARNET

model_info = [
    (4, "LinearNet_pcn4.pt", "PCN$_{4}$"),
    (8, "LinearNet_pcn8.pt", "PCN$_{8}$"),
    (16, "LinearNet_pcn16.pt", "PCN$_{16}$"),
    (32, "LinearNet_pcn32.pt", "PCN$_{32}$"),
    (None, "LinearNet_mlp.pt", "MLP"),
]

model_statistics: list[dict] = []

for ds_name in dataset_names:
    batch_size = 64
    in_features = 16 * 16 * 3
    out_features = 10 if ds_name == "cifar10" else 100

    train, test = None, None
    if ds_name == "cifar10":
        train = datasets.load_data(datasets.Cifar10("train", size=16), batch_size=batch_size)
        test = datasets.load_data(datasets.Cifar10("test", size=16), batch_size=batch_size)
    else:
        train = datasets.load_data(datasets.Cifar100("train", size=16), batch_size=batch_size)
        test = datasets.load_data(datasets.Cifar100("test", size=16), batch_size=batch_size)
    

    # load models from models_folder folder
    model_shape = [in_features, 1024, 1024, 2048, out_features]
    for d, fn, name in model_info:
        if "mlp" in fn:
            model = util.MLP(
                layers=model_shape,
            ).to(device)
        elif "pcn" in fn:
            model = util.PCN(
                layers=model_shape,
                dimensions=d,
            ).to(device)
        model.load_state_dict(torch.load(f"{models_folder}/{ds_name}/" + fn))

        print(f"Running {name} on {ds_name}...")

        model_statistics.append(
            {
                "Model Class": "LinearNet",
                "Dataset": ds_name,
                "Model": name,
                "# Convolutional Params (millions)": None,
                "# Linear Params (millions)": round(count_params(model) / 1e6, 3),
            } | run_epoch(model, train, test, device, in_features, flatten=True)
        )

df_linear = df = pd.DataFrame(model_statistics)

c:\Users\hette\anaconda3\envs\pytorch_cuda\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Running PCN$_{4}$ on cifar10...
Running PCN$_{8}$ on cifar10...
Running PCN$_{16}$ on cifar10...
Running PCN$_{32}$ on cifar10...
Running MLP on cifar10...
Running PCN$_{4}$ on cifar100...
Running PCN$_{8}$ on cifar100...
Running PCN$_{16}$ on cifar100...
Running PCN$_{32}$ on cifar100...
Running MLP on cifar100...


In [8]:

# CONVNET

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_names = ["cifar10", "cifar100"]
model_info = [
    (16, "ConvNet_pcn16.pt", "PCN$_{16}$"),
    (None, "ConvNet_mlp.pt", "MLP"),
]

model_statistics: list[dict] = []

for ds_name in dataset_names:
    batch_size = 64
    in_features = 3
    out_features = 10 if ds_name == "cifar10" else 100

    train, test = None, None
    if ds_name == "cifar10":
        train = datasets.load_data(datasets.Cifar10("train"), batch_size=batch_size)
        test = datasets.load_data(datasets.Cifar10("test"), batch_size=batch_size)
    else:
        train = datasets.load_data(datasets.Cifar100("train"), batch_size=batch_size)
        test = datasets.load_data(datasets.Cifar100("test"), batch_size=batch_size)

    # load models from models_folder folder
    conv_shape = [in_features, 32, 128, 512, 1024]
    linear_shape = [1024, out_features]
    for d, fn, name in model_info:
        if "mlp" in fn:
            model = ConvNet(
                conv_layers=conv_shape,
                linear_layers=linear_shape,
                linear_type="mlp"
            ).to(device)
        elif "pcn" in fn:
            model = ConvNet(
                conv_layers=conv_shape,
                linear_layers=linear_shape,
                linear_type="pcn",
                pcn_dims=d
            ).to(device)
        
        failed_load = False
        try:
            model.load_state_dict(torch.load(f"{models_folder}/{ds_name}/{fn}"))
        except:
            print(f"Failed to load {models_folder}/{ds_name}/{fn}")
            failed_load = True

        
        run_results = None
        if not failed_load:
            print(f"Running {name} on {ds_name}...")
            run_results = run_epoch(model, train, test, device, in_features)
        model_statistics.append(
            {
                "Model Class": "ConvNet",
                "Dataset": ds_name,
                "Model": name,
                "# Convolutional Params (millions)": round(count_params(model.net) / 1e6, 3),
                "# Linear Params (millions)": round(count_params(model.classifier) / 1e6, 3),
            } | (run_results if run_results is not None else {})
        )

df_conv = pd.DataFrame(model_statistics)

Running PCN$_{16}$ on cifar10...
Running MLP on cifar10...
Running PCN$_{16}$ on cifar100...
Running MLP on cifar100...


In [9]:

# ALEXNET

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_names = ["cifar10", "cifar100"]
model_info = [
    (16, "AlexNet_pcn16.pt", "PCN$_{16}$"),
    (None, "AlexNet_mlp.pt", "MLP"),
]

model_statistics: list[dict] = []

for ds_name in dataset_names:
    batch_size = 64
    in_features = 3
    out_features = 10 if ds_name == "cifar10" else 100

    train, test = None, None
    if ds_name == "cifar10":
        train = datasets.load_data(datasets.Cifar10("train", size=227), batch_size=batch_size)
        test = datasets.load_data(datasets.Cifar10("test", size=227), batch_size=batch_size)
    else:
        train = datasets.load_data(datasets.Cifar100("train", size=227), batch_size=batch_size)
        test = datasets.load_data(datasets.Cifar100("test", size=227), batch_size=batch_size)

    # load models from models_folder folder
    conv_shape = [in_features, 32, 128, 512, 1024]
    linear_shape = [1024, out_features]
    for d, fn, name in model_info:
        if "mlp" in fn:
            model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)
            model.classifier[6] = torch.nn.Linear(4096, out_features).to(device)
        elif "pcn" in fn:
            model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)
            model.classifier = util.PCN([9216, 4096, 4096, out_features], dimensions=d, dropout=0.5).to(device)
        
        failed_load = False
        try:
            model.load_state_dict(torch.load(f"{models_folder}/{ds_name}/{fn}"))
        except:
            print(f"Failed to load {models_folder}/{ds_name}/{fn}")
            failed_load = True

        
        run_results = None
        if not failed_load:
            print(f"Running {name} on {ds_name}...")
            run_results = run_epoch(model, train, test, device, in_features)
        model_statistics.append(
            {
                "Model Class": "AlexNet",
                "Dataset": ds_name,
                "Model": name,
                "# Convolutional Params (millions)": round(count_params(model.features) / 1e6, 3),
                "# Linear Params (millions)": round(count_params(model.classifier) / 1e6, 3),
            } | (run_results if run_results is not None else {})
        )

df_alex = pd.DataFrame(model_statistics)

c:\Users\hette\anaconda3\envs\pytorch_cuda\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Using cache found in C:\Users\hette/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\hette\anaconda3\envs\pytorch_cuda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the fu

Running PCN$_{16}$ on cifar10...


Using cache found in C:\Users\hette/.cache\torch\hub\pytorch_vision_v0.10.0


Running MLP on cifar10...


Using cache found in C:\Users\hette/.cache\torch\hub\pytorch_vision_v0.10.0


Running PCN$_{16}$ on cifar100...


Using cache found in C:\Users\hette/.cache\torch\hub\pytorch_vision_v0.10.0


Running MLP on cifar100...


In [10]:
pd.concat([df_linear, df_conv, df_alex]).to_csv('results.csv')